![](https://cbic.org.br/wp-content/uploads/2022/02/909fa5d1543363d809512626531cff03.jpeg)

# Projeto 001

Prof.: Nasser Santiago Boan

obs.: A situação descrita no projeto é ficticia e somente para fins educativos, não remete a operação real de nenhuma secretaria, servidor ou ente do MINFRA.

## Contexto
___

Você é o mais novo servidor da Secretaria Nacional de Trânsito (SENATRAN). Atualmente existe um esforço interno para medição da quantidade de processos, média de criação de novos processos por mês e sazonalidade da demanda de trabalho com o intuito de melhor dimensionar a equipe. Seu primeiro desafio é descobrir quantos processos sob responsabilidade do SENATRAN foram criados desde 1993 (somente os que estão no SEI), entender sazonalidade e tipos mais comuns. O resultado deste trabalho será avaliado pela sua liderança direta e afetará as decisões orçamentárias do próximo ano.

## Dados
___

Os dados para execução do projeto foram disponibilizados sob a política de dados abertos e podem ser encontrados [nesse link](https://dados.infraestrutura.gov.br/dadosabertos/SEI/Processos/). A função <code>get_data</code> foi criada para download dos dados (esse processo pode demorar alguns minutos, de acordo com a taxa de transferência da sua conexão).

## Métricas
___

Esse é um projeto de exploração das bases e servirá para tomada de decisão do tamanho do time. A métrica que determinará o sucesso do trabalho é a produtividade do time após a execução do redimensionamento, também podem ser consideradas as métricas de tempo médio de andamento dos processos, custo médio para execução de um processo entre outros. Porém, por uma questão de disponibilidade de dados essas métricas não serão calculadas.

## Planejamento
___

O trabalho pode ser dado como feito quando as seguintes perguntas estiverem respondidas:

1. Quantos processos sob responsabilidade do SENATRAN foram criados desde 1993?
2. Quantos processos sob responsabilidade do SENATRAN foram criados nos últimos 2 anos? Esse quantidade representa quanto do total?
3. Qual a média de criação de processos por mês sob responsabilidade do SENATRAN para cada ano onde existiu pelo menos 1 processo?
4. Em qual época do ano existe a maior criação de processos sob resposabilidade do SENATRAN?

Para esse projeto será considerado 'sob responsabilidade do SENATRAN' aquele processo que tiver a string 'SENATRAN' ou 'Senatran' em sua descrição.

## Código
___

### Importações

In [138]:
import pandas as pd
import os
from glob import glob
import requests as r

### Baixando e abrindo os dados

In [139]:
## a função faz o download dos dados caso já não existam na pasta 'dados/'

def get_data():
    if 'dados/processos.csv' not in glob('dados/*.csv'):
        
        dados = r.get('https://dados.infraestrutura.gov.br/dadosabertos/SEI/Processos/processos.csv').content
        
        with open('dados/processos.csv','wb') as file:
            file.write(dados)
    else:
        print("Os dados já foram anteriormente baixados e estão na pasta 'dados/'.")

In [140]:
get_data()

Os dados já foram anteriormente baixados e estão na pasta 'dados/'.


In [141]:
## abrindo os dados

df = pd.read_csv('dados/processos123.csv',sep=',',encoding='latin-1')

### Inspecionando os dados

In [142]:
df.head()

,id_processo,numero_processo,id_tipo_processo,tipo_processo,data_criacao_processo
0,5463334,35238.001398/1993-66,100000228,Pessoal: Pensão por Morte de Servidor,05/05/1993
1,603412,50000.004633/1993-47,100000228,Pessoal: Pensão por Morte de Servidor,06/05/1993
2,296943,50000.004716/1993-72,100000228,Pessoal: Pensão por Morte de Servidor,07/05/1993
3,1170920,10380.003879/1993-87,100000228,Pessoal: Pensão por Morte de Servidor,07/05/1993
4,1671908,50000.004681/1993-81,100000121,Pessoal: Aposentadoria - Pensão Temporária,07/05/1993


In [143]:
df.shape

(671966, 5)

In [144]:
df.numero_processo.nunique() == df.shape[0]

True

O dataset possui exatas 671.966 entradas e cinco colunas e não existem dois processos com o mesmo número.

In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671966 entries, 0 to 671965
Data columns (total 5 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   id_processo            671966 non-null  int64 
 1   numero_processo        671966 non-null  object
 2   id_tipo_processo       671966 non-null  int64 
 3   tipo_processo          671966 non-null  object
 4   data_criacao_processo  671966 non-null  object
dtypes: int64(2), object(3)
memory usage: 25.6+ MB


### Tratando a coluna de data

In [146]:
df['data_criacao_processo'] = pd.to_datetime(df.data_criacao_processo,format='%d/%m/%Y')

In [147]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671966 entries, 0 to 671965
Data columns (total 5 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   id_processo            671966 non-null  int64         
 1   numero_processo        671966 non-null  object        
 2   id_tipo_processo       671966 non-null  int64         
 3   tipo_processo          671966 non-null  object        
 4   data_criacao_processo  671966 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 25.6+ MB


### Analisando os dados

1. Quantos processos sob responsabilidade do SENATRAN foram criados desde 1993?
___

In [148]:
dados_senatran = df[(df.tipo_processo.str.contains('SENATRAN')) | (df.tipo_processo.str.contains('Senatran'))].copy()
dados_senatran.head()

,id_processo,numero_processo,id_tipo_processo,tipo_processo,data_criacao_processo
589,2001429,80001.014528/2004-27,100000644,Senatran: Análise de Normas do Contran,2004-09-09
702,1732542,08656.094753/2016-41,100000611,Senatran: Infrações: Recursos de Multa,2016-08-12
703,1709963,08659.015467/2012-73,100000611,Senatran: Infrações: Recursos de Multa,2012-08-29
705,1781075,08654.009504/2016-88,100000611,Senatran: Infrações: Recursos de Multa,2016-08-12
712,2283483,08656.095649/2016-73,100000643,Senatran: Recurso de Multa,2016-08-13


In [149]:
dados_senatran.data_criacao_processo.min(),dados_senatran.data_criacao_processo.max()

(Timestamp('2001-02-01 00:00:00'), Timestamp('2022-09-30 00:00:00'))

In [150]:
qtd_de_processos_total = dados_senatran.numero_processo.count()

print(f'Desde 1993 foram criados, exatamente {qtd_de_processos_total} processos sob responsabilidade do SENATRAN')

Desde 1993 foram criados, exatamente 46704 processos sob responsabilidade do SENATRAN


2. Quantos processos sob responsabilidade do SENATRAN foram criados nos últimos 2 anos? Esse quantidade representa quanto do total?
___

In [151]:
qtd_de_processos_2_anos = dados_senatran[dados_senatran.data_criacao_processo >= '2020-01-01'].numero_processo.count()
qtd_de_processos_2_anos

28577

In [152]:
porcentagem = qtd_de_processos_2_anos / qtd_de_processos_total
porcentagem

0.6118747858855772

In [153]:
print(f'Desde 2020 o SENATRAN criou {qtd_de_processos_2_anos} processos, representando {round(porcentagem*100,2)}% do total desde sua instituição.')

Desde 2020 o SENATRAN criou 28577 processos, representando 61.19% do total desde sua instituição.


3. Qual a média de criação de processos por mês sob responsabilidade do SENATRAN para cada ano onde existiu pelo menos 1 processo?
___

In [154]:
dados_senatran.head()

,id_processo,numero_processo,id_tipo_processo,tipo_processo,data_criacao_processo
589,2001429,80001.014528/2004-27,100000644,Senatran: Análise de Normas do Contran,2004-09-09
702,1732542,08656.094753/2016-41,100000611,Senatran: Infrações: Recursos de Multa,2016-08-12
703,1709963,08659.015467/2012-73,100000611,Senatran: Infrações: Recursos de Multa,2012-08-29
705,1781075,08654.009504/2016-88,100000611,Senatran: Infrações: Recursos de Multa,2016-08-12
712,2283483,08656.095649/2016-73,100000643,Senatran: Recurso de Multa,2016-08-13


In [155]:
dados_senatran['ano'] = dados_senatran.data_criacao_processo.dt.year

dados_senatran.head()

,id_processo,numero_processo,id_tipo_processo,tipo_processo,data_criacao_processo,ano
589,2001429,80001.014528/2004-27,100000644,Senatran: Análise de Normas do Contran,2004-09-09,2004
702,1732542,08656.094753/2016-41,100000611,Senatran: Infrações: Recursos de Multa,2016-08-12,2016
703,1709963,08659.015467/2012-73,100000611,Senatran: Infrações: Recursos de Multa,2012-08-29,2012
705,1781075,08654.009504/2016-88,100000611,Senatran: Infrações: Recursos de Multa,2016-08-12,2016
712,2283483,08656.095649/2016-73,100000643,Senatran: Recurso de Multa,2016-08-13,2016


In [127]:
dados_senatran.groupby('ano').count().id_processo / 12

ano
2001       1.083333
2002       0.166667
2003       0.333333
2004       0.666667
2005       0.833333
2006       1.666667
2007       1.416667
2008       1.416667
2009       1.666667
2010       2.000000
2011       4.000000
2012       4.500000
2013       5.666667
2014       8.000000
2015      14.500000
2016      55.000000
2017      65.333333
2018      35.333333
2019    1307.000000
2020     797.333333
2021     851.583333
2022     732.500000
Name: id_processo, dtype: float64

4. Em qual época do ano existe a maior criação de processos sob resposabilidade do SENATRAN?
___

In [135]:
dados_senatran.head()

,id_processo,numero_processo,id_tipo_processo,tipo_processo,data_criacao_processo,ano,month
589,2001429,80001.014528/2004-27,100000644,Senatran: Análise de Normas do Contran,2004-09-09,2004,9
702,1732542,08656.094753/2016-41,100000611,Senatran: Infrações: Recursos de Multa,2016-08-12,2016,8
703,1709963,08659.015467/2012-73,100000611,Senatran: Infrações: Recursos de Multa,2012-08-29,2012,8
705,1781075,08654.009504/2016-88,100000611,Senatran: Infrações: Recursos de Multa,2016-08-12,2016,8
712,2283483,08656.095649/2016-73,100000643,Senatran: Recurso de Multa,2016-08-13,2016,8


In [136]:
dados_senatran['month'] = dados_senatran.data_criacao_processo.dt.month

dados_senatran.head()

,id_processo,numero_processo,id_tipo_processo,tipo_processo,data_criacao_processo,ano,month
589,2001429,80001.014528/2004-27,100000644,Senatran: Análise de Normas do Contran,2004-09-09,2004,9
702,1732542,08656.094753/2016-41,100000611,Senatran: Infrações: Recursos de Multa,2016-08-12,2016,8
703,1709963,08659.015467/2012-73,100000611,Senatran: Infrações: Recursos de Multa,2012-08-29,2012,8
705,1781075,08654.009504/2016-88,100000611,Senatran: Infrações: Recursos de Multa,2016-08-12,2016,8
712,2283483,08656.095649/2016-73,100000643,Senatran: Recurso de Multa,2016-08-13,2016,8


In [137]:
dados_senatran.groupby('month').id_processo.count()

month
1     2805
2     3505
3     5273
4     4183
5     4593
6     3963
7     4486
8     4850
9     4332
10    3271
11    2909
12    2534
Name: id_processo, dtype: int64

### Funções utilizadas

<code>.read_csv()    -> https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html</code><br>
<code>.head()        -> https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html</code><br>
<code>.shape         -> https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shape.html</code><br>
<code>.nunique()     -> https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.nunique.html</code><br>
<code>.info()        -> https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html</code><br>
<code>.to_datetime() -> https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html</code><br>
<code>.min()         -> https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.min.html</code><br>
<code>.max()         -> https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.max.html</code><br>
<code>.count()       -> https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.count.html</code><br>
<code>count()        -> https://docs.python.org/3/library/functions.html#round</code><br>
<code>.dt.year()     -> https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.year.html</code><br>
<code>.groupby()     -> https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html</code><br>


